In [1]:
import pandas as pd  # pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##  Random Forest
példa:https://www.codementor.io/@agarrahul01/multiclass-classification-using-random-forest-on-scikit-learn-library-hkk4lwawu

Csak numerikus targetváltozók

In [2]:
#Importing Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn import metrics


/home/akos/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
df_train = pd.read_csv("../datasets/train1.csv", sep=",")

In [4]:
df_train.head()

,Unnamed: 0,device_id,events_num,min_timestamp,max_timestamp,events_per_day,label_548,label_i_548,label_704,label_i_704,...,label_787,label_i_787,label_22222,label_i_22222,gender,age,group,phone_brand,device_model,phone_brand_eng
0,0,-8260683887967679142,1,2016-05-01 14:23:37,2016-05-01 14:23:37,0.000000,5,30,3,24,...,0,0,4,102,M,35,M32-38,小米,MI 2,Xiaomi
1,1,7477216237379271436,3,2016-05-04 14:08:17,2016-05-06 18:51:15,0.065502,31,29,25,17,...,3,0,27,78,F,37,F33-42,华为,荣耀6 plus,Huawei
2,2,6352067998666467520,11,2016-05-03 22:14:30,2016-05-05 17:27:58,0.072820,70,34,57,19,...,11,0,211,103,M,32,M32-38,华为,荣耀畅玩4X,Huawei
3,3,1508636020748379883,5,2016-05-01 12:41:38,2016-05-05 12:34:15,0.198975,50,39,38,36,...,2,3,69,80,F,28,F27-28,华为,荣耀畅玩4X,Huawei
4,4,-6876541075223249434,6,2016-05-01 02:51:41,2016-05-06 16:30:01,0.094715,63,187,46,162,...,0,0,126,520,M,75,M39+,魅族,魅蓝NOTE,Meizu


In [5]:
#faktorizalas
factor = pd.factorize(df_train['group'])
factor

(array([0, 1, 0, ..., 8, 0, 4]),
 Index(['M32-38', 'F33-42', 'F27-28', 'M39+', 'M23-26', 'M27-28', 'M29-31',
        'M22-', 'F43+', 'F24-26', 'F23-', 'F29-32'],
       dtype='object'))

In [6]:
df_train.group = factor[0]
definitions = factor[1]

In [7]:
factor = pd.factorize(df_train['phone_brand_eng'])
df_train.phone_brand_eng = factor[0]
definitions_phone_brand_eng = factor[1]
factor = pd.factorize(df_train['device_model'])
df_train.device_model = factor[0]
definitions_device_model = factor[1]

In [8]:
df_train.head()

,Unnamed: 0,device_id,events_num,min_timestamp,max_timestamp,events_per_day,label_548,label_i_548,label_704,label_i_704,...,label_787,label_i_787,label_22222,label_i_22222,gender,age,group,phone_brand,device_model,phone_brand_eng
0,0,-8260683887967679142,1,2016-05-01 14:23:37,2016-05-01 14:23:37,0.000000,5,30,3,24,...,0,0,4,102,M,35,0,小米,0,0
1,1,7477216237379271436,3,2016-05-04 14:08:17,2016-05-06 18:51:15,0.065502,31,29,25,17,...,3,0,27,78,F,37,1,华为,1,1
2,2,6352067998666467520,11,2016-05-03 22:14:30,2016-05-05 17:27:58,0.072820,70,34,57,19,...,11,0,211,103,M,32,0,华为,2,1
3,3,1508636020748379883,5,2016-05-01 12:41:38,2016-05-05 12:34:15,0.198975,50,39,38,36,...,2,3,69,80,F,28,2,华为,2,1
4,4,-6876541075223249434,6,2016-05-01 02:51:41,2016-05-06 16:30:01,0.094715,63,187,46,162,...,0,0,126,520,M,75,3,魅族,3,2


In [9]:
## TODO : nan, infiniti -> 0
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)

In [10]:
magyarazo_valtozok = []
for m in df_train.columns:
    if m not in ['device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        magyarazo_valtozok += [m]
X_train, X_test, y_train, y_test = train_test_split(df_train[magyarazo_valtozok], df_train['group'], test_size=0.1,random_state=109) # 90% training and 10% test


### Feature scaling
This is a very important step in machine learning. It helps the algorithm quickly learn a better solution to the problem.

We will use a standard scaler provided in the sklearn library. It subtracts the mean value of the observation and then divides it by the unit variance of the observation.
We will perform the following steps:

- Define a scaler by calling the function from sklearn library.
- Transform train feature dataset (X_train) and fit the scaler on train feature dataset.
- Use the scaler to transform test feature dataset (X_test).

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [13]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.1858974358974359
